In [1]:
#Based on the code of Samantha Molnar found on
#http://samanthamolnar.me/personal/2017/05/02/hacking-google-trends.html

from pytrends.request import TrendReq
import pandas as pd
import numpy as np
import math
import time
import random
keyword_list = ["bitcoin"] # term to search in googletrends
dates0=pd.read_excel("dates0.xlsx") #this file contains the list of weeks to be requested to google
#as google raises error code 500 after 100 requests, there is another file "dates1.xlsx" with the last weeks
dates1=pd.read_excel("dates1.xlsx")

In [2]:
dates1

,Weeks
0,2019-05-18 2019-05-25
1,2019-05-25 2019-06-01
2,2019-06-01 2019-06-08
3,2019-06-08 2019-06-15
4,2019-06-15 2019-06-21


In [3]:
dfs0={}
for i in dates0:
    dfs0[i]={}
    for j in dates0[i]: #each week in the year
        if type(j) == float and math.isnan(j):
            continue
        endpoints = j.split(" ")
        key = endpoints[0] + "T00 " + endpoints[1] + "T00"
        time.sleep(random.randint(0,2))
        p=TrendReq()
        p.build_payload(keyword_list,timeframe=key)
        dfs0[i][j]=p.interest_over_time()
        print("obtained google trends for the period {}".format(key))
#saving the first 98 weeks trend scores in a dictionary(file)
np.save('dfs0.npy', dfs0) 

obtained google trends for the period 2017-07-01T00 2017-07-08T00
obtained google trends for the period 2017-07-08T00 2017-07-15T00
obtained google trends for the period 2017-07-15T00 2017-07-22T00
obtained google trends for the period 2017-07-22T00 2017-07-29T00
obtained google trends for the period 2017-07-29T00 2017-08-05T00
obtained google trends for the period 2017-08-05T00 2017-08-12T00
obtained google trends for the period 2017-08-12T00 2017-08-19T00
obtained google trends for the period 2017-08-19T00 2017-08-26T00
obtained google trends for the period 2017-08-26T00 2017-09-02T00
obtained google trends for the period 2017-09-02T00 2017-09-09T00
obtained google trends for the period 2017-09-09T00 2017-09-16T00
obtained google trends for the period 2017-09-16T00 2017-09-23T00
obtained google trends for the period 2017-09-23T00 2017-09-30T00
obtained google trends for the period 2017-09-30T00 2017-10-07T00
obtained google trends for the period 2017-10-07T00 2017-10-14T00
obtained g

In [4]:
dfs1={}
for i in dates1:
    dfs1[i]={}
    for j in dates1[i]: #each week in the year
        if type(j) == float and math.isnan(j):
            continue
        endpoints = j.split(" ")
        key = endpoints[0] + "T00 " + endpoints[1] + "T00"
        time.sleep(2)
        p=TrendReq()
        p.build_payload(keyword_list,timeframe=key)
        dfs1[i][j]=p.interest_over_time()
        print("obtained Gtrends for the period {}".format(key))

obtained Gtrends for the period 2019-05-18T00 2019-05-25T00
obtained Gtrends for the period 2019-05-25T00 2019-06-01T00
obtained Gtrends for the period 2019-06-01T00 2019-06-08T00
obtained Gtrends for the period 2019-06-08T00 2019-06-15T00
obtained Gtrends for the period 2019-06-15T00 2019-06-21T00


In [5]:
#saving other weeks trend scores in other dictionary
np.save('dfs1.npy', dfs1) 
##What Google give us is the percentage of how much the keyword has been searched, calculated over that week

In [7]:
#loading and Merging both dicts.
dfs0=np.load('dfs0.npy').item()
dfs1=np.load('dfs1.npy').item()
dfs={}
dfs.update(dfs0)
dfs["Weeks"].update(dfs1["Weeks"])
len(dfs["Weeks"])==len(dates0)+len(dates1)

True

In [8]:
#so we have to "standarize" all weeks by Stitching  together the timeseries: 
#we can calculate the scaling ratio between two different intervals with its last score and the first of the next interval,
#and then scale one of those intervals to match the other.
keyword=keyword_list#="bitcoin"
dates=pd.concat([dates0,dates1]) #joining the dates DFs
timeseries=pd.DataFrame(dfs["Weeks"]["2017-07-01 2017-07-08"]) # the initial timeseries as DF
for i in dates:
    for j in dates[i]:
        if type(j) == float and math.isnan(j):
            continue
        if j != "2017-07-01 2017-07-08":
            df1 = timeseries
            df2 = dfs[i][j]
            ratio = float(df2[keyword].iloc[0])/float(df1[keyword].iloc[-1]) 
            df1[keyword]=ratio*df1[keyword]
            timeseries = pd.concat([timeseries,df2])
m=np.max(timeseries[keyword])
timeseries[keyword]=100*timeseries[keyword]/m
del timeseries['isPartial']
timeseries.tail()

,bitcoin
date,
2019-06-20 20:00:00,7.514694
2019-06-20 21:00:00,7.436416
2019-06-20 22:00:00,7.749528
2019-06-20 23:00:00,6.731913
2019-06-21 00:00:00,6.262245


In [9]:
# Export the final timeseries with the standarized Gtrends to Excel file
timeseries.to_excel("bitcoingtrends.xlsx",sheet_name='Gtrends')
print("file generated: bitcoingtrends.xlsx")